In [ ]:
import numpy as np
import pandas as pd
from train import main

In [ ]:
import argparse

args = {'model': 'BiLSTM', 'features': '../hh_dataset/hh_npy/fib_hh102_feature.npy',
        'activities': '../hh_dataset/hh_npy/fib_hh102_activity.npy', 
        'feature_encoding': '1d_cnn', 
        'delta': 10}
args = [f"--{k}={v}" for k, v in args.items()]
p = argparse.ArgumentParser()
p.add_argument('--model', dest='model', action='store', default='', help='deep model')
p.add_argument('--features', action='store', default='', required=True, help='deep model')
p.add_argument('--activities', action='store', default='', required=True, help='deep model')
p.add_argument('--feature_encoding', action='store', default='mean_with_weight', required=True, help='deep model')
p.add_argument('--delta', action='store', default='20', required=False, help='deep model')
args = p.parse_args(args)

models = main(args)

In [ ]:
models1 = deepcopy(models)

In [ ]:
def calculate_metrics(pred, target, threshold=0.3):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }

X_train, X_test, y_train, y_test = processed_features[train], processed_features[test], activities[train], activities[test]

batch_size = 32
n_iters = 100

train_features, train_labels = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
dev_features, dev_labels = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

train_dataset = DataLoader(ActivityDataset(train_features, train_labels), batch_size=batch_size, shuffle=False)
dev_dataset = DataLoader(ActivityDataset(dev_features, dev_labels), batch_size=len(y_test), shuffle=False)

for features, labels in tqdm(dev_dataset, total=len(dev_dataset)):
# train_labels, train_tensor = torch.tensor(train_activities, dtype=torch.float32), torch.tensor(train_features, dtype=torch.float32)
# dev_labels, dev_tensor = torch.tensor(dev_activities, dtype=torch.float32), torch.tensor(dev_features, dtype=torch.float32)

    features = features.to(device).float()
    labels = labels.to(device).float()

    outputs = torch.Tensor([])
    for model in models:
        output, logits = model(features)
        # print(output.shape)
        outputs = torch.cat((outputs, output.detach().cpu()), dim=-1)

    # print(outputs.shape)
    # print(labels.shape)
    pred = outputs.numpy().round()
    result = calculate_metrics(pred, labels.cpu().numpy())
    print(result)

In [ ]:
print(pred.shape)
print(labels.shape)

In [ ]:
def multi_label_train(X_train, X_test, y_train, y_test, models):
    batch_size = 16
    n_iters = 100
    total_classes = np.sum(y_train.T, axis=1)
    n_categories = len(y_train.T) - 1
    
    medians = [total_classes[k]/len(y_train) for k in range(0,n_categories)]
    median_all = np.mean(medians)
    my_freqs = median_all/(np.float64(y_train)+(10E-14))

    model = Multi(models, len(models), len(models))
    learning_rate = 0.0001
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)
    criterion = FocalLoss(torch.tensor(my_freqs).to(device))

    train_features, train_labels = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    dev_features, dev_labels = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    for iter in trange(1, n_iters + 1):
        train_dataset = DataLoader(ActivityDataset(train_features, train_labels), batch_size=batch_size, shuffle=False)
        dev_dataset = DataLoader(ActivityDataset(dev_features, dev_labels), batch_size=len(y_test), shuffle=False)
        model.train()
        training_loss = 0
        best_f1_score = 0
        best_classification_report = None

        for features, labels in tqdm(train_dataset, total=len(train_dataset)):
        # train_labels, train_tensor = torch.tensor(train_activities, dtype=torch.float32), torch.tensor(train_features, dtype=torch.float32)
        # dev_labels, dev_tensor = torch.tensor(dev_activities, dtype=torch.float32), torch.tensor(dev_features, dtype=torch.float32)

            features = features.to(device).float()
            labels = labels.to(device).float()

            output, logits = model(features)
            # loss = criterion(logits, labels)
            pred = logits.cpu().detach().numpy().round()
            result = calculate_metrics(pred, labels.cpu().numpy())

            # loss.backward()
            # optimizer.step()
            # optimizer.zero_grad()
            
            validation_loss = None
            #scheduler.step()
            current_loss = 0
            
        for dev_tensor, dev_activities in dev_dataset:
            dev_tensor = dev_tensor.to(device)
            dev_labels = dev_activities.to(device)
            if len(dev_labels.shape) < 2:
                dev_labels = dev_labels.unsqueeze(1)

            model.eval()
            with torch.no_grad():
                prediction, logits = model(dev_tensor)
                
                pred = logits.cpu().detach().numpy().round()

                # validation_loss = criterion(logits, dev_labels)
                result = calculate_metrics(pred, dev_activities.cpu().numpy())
                
                #backprop
                if result['micro/f1'] > best_f1_score:
                    best_f1_score = result['micro/f1']
                    best_classification_report = result
                    best_epoch = iter
                    best_validation_loss = validation_loss
                validation_loss = 0
                
            if iter%print_every == 0:
                accur.append(result)
                
                # result_file.write("epoch {}\tloss : {}\t accuracy : {}\n".format(iter,loss,acc))
                print("epoch {}\tloss : {}\t accuracy : {}".format(iter,validation_loss,result))
                # print(classification_report(dev_activities, pred, digits=4))
        
        # current_loss += validation_loss.item()
        # if early_stopper.early_stop(validation_loss):
        #     print("epoch {}\tloss : {}\t accuracy : {}".format(best_epoch,best_validation_loss,best_classification_report))
        #     # result_file.write("Best epoch at " + str(best_epoch) + ". Early stopping at epoch " + str(iter))
        #     break

        if iter % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
        # result_file.write(str(best_classification_report) + '\n')
        # return pd.DataFrame(best_classification_report).round(4)
        
        # ensemble_model_results = multi_label_train(X_train, X_test, y_train, y_test, models)
multi_label_train(X_train, X_test, y_train, y_test, models)

In [40]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import time
import math
import sys
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy
import tqdm

from numpy.typing import NDArray
from over_sample import get_minority_instace, MLSMOTE
from model import LSTM, EarlyStopper, LSTM_1d, Multi_out_LSTM, FocalLoss
import json
import warnings
warnings.filterwarnings('ignore')
import os

import argparse
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, TimeSeriesSplit, train_test_split
from tqdm import tqdm, trange

device = "cuda:0"
class Multi(nn.Module):

    def __init__(self, models, input_dim,output_dim):
        super(Multi, self).__init__()
        self.models = models
        self.out = nn.Sequential(
            # nn.Dropout(p=0.2),
            nn.Linear(in_features=input_dim, out_features=output_dim)
        )
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, inputs):
        inputs = inputs.to(device)
        outputs = torch.Tensor([]).to(device)
        for model in models:
            model = model.to(device)
            output, logits = model(inputs)
            # print(output.shape)
            outputs = torch.cat((outputs, logits), dim=-1)
        # self.out = self.out.to(device)
        # logits = self.out(outputs)
        return self.sigmoid(outputs), outputs
    
def calculate_metrics(pred, target, threshold=0.3):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }

class ActivityDataset(Dataset):
  """
  This is a custom dataset class. It can get more complex than this, but simplified so you can understand what's happening here without
  getting bogged down by the preprocessing
  """
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
    if len(self.X) != len(self.Y):
      raise Exception("The length of X does not match the length of Y")

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    # note that this isn't randomly selecting. It's a simple get a single item that represents an x and y
    _x = self.X[index]
    _y = self.Y[index]

    return _x, _y
  
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

features = np.load(args.features)
activities = np.load(args.activities)

mapping_file = args.features.rsplit('_', 1)[0] + '_mapping.json'
with open(mapping_file, 'r') as f:
    activitiy_mapping = json.load(f)
ensemble_activities = activities.T

processed_features = features

input_size = processed_features.shape[-1]

n_hidden = 512
n_categories = len(activitiy_mapping) - 1
n_layer = 3

#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)

n_iters = 1000
print_every = n_iters // 200
plot_every =  n_iters // 200
batch_size = 32

print_every = 1
tsv = TimeSeriesSplit(n_splits=3)
kfold = KFold(n_splits=3)
# Keep track of losses for plotting
current_loss = 0
all_losses = []
accur = []
train, test = list(tsv.split(processed_features, activities))[-1]

X_train, X_test, y_train, y_test = processed_features[train], processed_features[test], activities[train], activities[test]

def multi_label_train(X_train, X_test, y_train, y_test, models):
    batch_size = 16
    n_iters = 100
    total_classes = np.sum(activities.T, axis=1)

    medians = [total_classes[k]/len(activities) for k in range(0,n_categories)]
    median_all = np.mean(medians)
    my_freqs = median_all/(np.float64(medians)+(10E-14))

    model = Multi(models, len(models), len(models))
    learning_rate = 0.0001
    optimizer = optim.Adam(model.parameters(),lr=learning_rate)
    criterion = FocalLoss(torch.tensor(my_freqs).to(device))

    train_features, train_labels = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
    dev_features, dev_labels = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
    
    for iter in trange(1, n_iters + 1):
        train_dataset = DataLoader(ActivityDataset(train_features, train_labels), batch_size=batch_size, shuffle=False)
        dev_dataset = DataLoader(ActivityDataset(dev_features, dev_labels), batch_size=len(y_test), shuffle=False)
        model.train()
        training_loss = 0
        best_f1_score = 0
        best_classification_report = None

        for features, labels in tqdm(train_dataset, total=len(train_dataset)):
        # train_labels, train_tensor = torch.tensor(train_activities, dtype=torch.float32), torch.tensor(train_features, dtype=torch.float32)
        # dev_labels, dev_tensor = torch.tensor(dev_activities, dtype=torch.float32), torch.tensor(dev_features, dtype=torch.float32)

            features = features.to(device).float()
            labels = labels.to(device).float()

            output, logits = model(features)
            loss = criterion(logits, labels)
            pred = output.cpu().detach().numpy().round()
            result = calculate_metrics(pred, labels.cpu().numpy())

            # loss.backward()
            # optimizer.step()
            # optimizer.zero_grad()
            
            validation_loss = None
            #scheduler.step()
            current_loss = 0
            
        for dev_tensor, dev_activities in dev_dataset:
            dev_tensor = dev_tensor.to(device)
            dev_labels = dev_activities.to(device)
            if len(dev_labels.shape) < 2:
                dev_labels = dev_labels.unsqueeze(1)

            model.eval()
            with torch.no_grad():
                prediction, logits = model(dev_tensor)
                
                pred = prediction.cpu().detach().numpy().round()

                validation_loss = criterion(logits, dev_labels)
                result = calculate_metrics(pred, dev_activities.cpu().numpy())
                
                #backprop
                if result['micro/f1'] > best_f1_score:
                    best_f1_score = result['micro/f1']
                    best_classification_report = result
                    best_epoch = iter
                    best_validation_loss = validation_loss

            if iter%print_every == 0:
                accur.append(result)
                
                # result_file.write("epoch {}\tloss : {}\t accuracy : {}\n".format(iter,loss,acc))
                print("epoch {}\tloss : {}\t accuracy : {}".format(iter,validation_loss,result))
                # print(classification_report(dev_activities, pred, digits=4))
        
        current_loss += validation_loss.item()
        # if early_stopper.early_stop(validation_loss):
        #     print("epoch {}\tloss : {}\t accuracy : {}".format(best_epoch,best_validation_loss,best_classification_report))
        #     # result_file.write("Best epoch at " + str(best_epoch) + ". Early stopping at epoch " + str(iter))
        #     break

        if iter % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
        # result_file.write(str(best_classification_report) + '\n')
        # return pd.DataFrame(best_classification_report).round(4)
        
        # ensemble_model_results = multi_label_train(X_train, X_test, y_train, y_test, models)
multi_label_train(X_train, X_test, y_train, y_test, models)

  1%|          | 1/100 [00:09<15:35,  9.45s/it]

epoch 1	loss : 0.015425018034875393	 accuracy : {'micro/precision': 0.8050036791758646, 'micro/recall': 0.812778603268945, 'micro/f1': 0.8088724584103512, 'macro/precision': 0.774747322385416, 'macro/recall': 0.7950185878400265, 'macro/f1': 0.7801323493842599, 'samples/precision': 0.334453907067085, 'samples/recall': 0.33687434633198865, 'samples/f1': 0.3302572694997616}


  2%|▏         | 2/100 [00:18<15:26,  9.45s/it]

epoch 2	loss : 0.015425018034875393	 accuracy : {'micro/precision': 0.8050036791758646, 'micro/recall': 0.812778603268945, 'micro/f1': 0.8088724584103512, 'macro/precision': 0.774747322385416, 'macro/recall': 0.7950185878400265, 'macro/f1': 0.7801323493842599, 'samples/precision': 0.334453907067085, 'samples/recall': 0.33687434633198865, 'samples/f1': 0.3302572694997616}


  2%|▏         | 2/100 [00:21<17:46, 10.88s/it]


KeyboardInterrupt: 

In [ ]:
result2file = {'original': 'BiLSTM-original_feature.tsv', 
'Fib_FTW': 'BiLSTM-fib_ftw.tsv', 
'mean_feature':'BiLSTM-22-01-23-18:42:59 (mean).tsv',
'mean_std':'BiLSTM-22-01-23-19:05:49(mean_std).tsv',
'mean_with_weight':'BiLSTM-22-01-23-22:07:31(mean_with_weight).tsv',
'10mins_feature': 'BiLSTM-22-01-23-22:24:13(10_mins_features).tsv',
'5mins_feature':'BiLSTM-23-01-23-01:18:51(5_mins_features).tsv',
'fib_ftw_feature':'BiLSTM-23-01-23-02:51:59(fib_ftw_features).tsv',
'fib_ftw_5mins_feature':'BiLSTM-fib-ftw-5mins.tsv',
'fib_ftw_2mins_feature':'BiLSTM-fib-ftw-2mins.tsv'}

In [ ]:
from collections import defaultdict
activity_pos_detail = {}
activity_marco_detail = {}
overall = defaultdict(dict)
present_results = ['mean_with_weight', 'fib_ftw_feature', 'fib_ftw_5mins_feature', 'fib_ftw_2mins_feature']
for result in present_results:
    csv = pd.read_csv('./result/' + result2file[result], sep='\t').set_index('activity')
    activity_pos_detail[result] = csv['positive_f1']
    activity_marco_detail[result] = csv['marco_f1']
    if 'Mean' not in csv.index:
        overall[result]['positive_f1'] = csv['positive_f1'].astype(float).mean(axis=0)
        overall[result]['marco_f1'] = csv['marco_f1'].astype(float).mean(axis=0)
    else:
        for col, average in csv[['positive_f1', 'marco_f1']].loc['Mean'].items():
            overall[result][col] = average

In [ ]:
print(pd.DataFrame(overall).T.to_html())

In [ ]:
print(pd.DataFrame(overall).T.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>mean_feature</th>
      <th>mean_std</th>
      <th>mean_with_weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>positive_f1</th>
      <td>0.6020</td>
      <td>0.5580</td>
      <td>0.6533</td>
    </tr>
    <tr>
      <th>marco_f1</th>
      <td>0.7911</td>
      <td>0.7669</td>
      <td>0.8169</td>
    </tr>
  </tbody>
</table>

In [ ]:
import os
features = {}
activities = {}

for dir, _, files in list(os.walk('../hh_dataset/hh_npy')):
    for file in files:
        # print(file)
        npy_file = np.load(os.path.join(dir, file))
        if 'feature' in file:
            features[file] = npy_file.shape
        else:
            activities[file] = npy_file.shape


In [ ]:
print(features)
# print(activities)

In [ ]:
import numpy as np
lamd = 0.3
denom = [np.exp(-lamd * i) for i in range(10)]
denom

In [ ]:
import numpy as np
import pandas as pd
pred = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weight/predictions.npy', allow_pickle=True)
act = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weight/activities.npy', allow_pickle=True)
over_sample_pred = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weight_oversample/predictions.npy', allow_pickle=True)
over_sample_act = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weight_oversample/activities.npy', allow_pickle=True)
cnn_pred = np.load('./result_new/BiLSTM-(fib_hh102)_1d_cnn_oversample/predictions.npy', allow_pickle=True)
cnn_act = np.load('./result_new/BiLSTM-(fib_hh102)_1d_cnn_oversample/activities.npy', allow_pickle=True)

cnn_pred = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weightwithout_act_mapping/predictions.npy', allow_pickle=True)
cnn_act = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weightwithout_act_mapping/activities.npy', allow_pickle=True)
pred.shape

In [ ]:
act[-1115:].shape

In [ ]:
activitiy_mapping = {'Sleep': 0,
    'Bed_Toilet_Transition': 1,
    'Toilet': 2,
    'Take_Medicine': 3,
    'Dress': 4,
    'Work': 5,
    'Cook': 6,
    'Eat': 7,
    'Wash_Dishes': 8,
    'Relax': 9,
    'Personal_Hygiene': 10,
    'Bathe': 11,
    'Groom': 12,
    'Drink': 13,
    'Leave_Home': 14,
    'Enter_Home': 15,
    'Phone': 16,
    'Other_Activity': 17}

In [ ]:
from collections import Counter
Counter(pred[0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
from sklearn.metrics import multilabel_confusion_matrix
def calculate_metrics(pred, target, threshold=0.0):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='weighted'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='weighted'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='weighted'),
            }

In [ ]:
calculate_metrics(pred.T, act[-1115:])

In [33]:
import torch
from torch import nn
import numpy as np
import math

def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], -1)

class SineActivation(nn.Module):
    def __init__(self, batch_size, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        # torch.Tensor.fill_(self.w, torch.pi * 2 / 7)
        # torch.Tensor.fill_(self.w0, torch.pi * 2 / 7)
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        # tau.shape = (1, in_features)
        # w.shape = (in_features, out_features-1)
        # w0.shape = (in_features, 1)
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

class CosineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(CosineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.cos

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [38]:
batch_size = 10
out_features = 56
sineact = SineActivation(batch_size, 1, out_features)
# cosact = CosineActivation(1, 64)

# print(sineact(torch.Tensor([[5, 10]])).shape)
a = torch.rand(batch_size, out_features)
time = torch.tensor([[1, 1]], dtype=torch.float).reshape(-1, 1)
print(sineact(torch.Tensor(time)).shape)
# # np.array([[5, 10]]).shape
sineact(torch.Tensor(time))
time.shape

torch.Size([2, 56])


torch.Size([2, 1])

In [26]:
torch.Tensor.fill_(torch.zeros(1, out_features), torch.pi * 2 / 7)

tensor([[0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976, 0.8976,
         0.8976, 0.8976]])

In [ ]:
calculate_metrics(over_sample_pred.T, over_sample_act[-1115:])

In [ ]:
calculate_metrics(cnn_pred.T, cnn_act[-1115:])


In [ ]:
import json
with open('../hh_dataset/hh_npy/fib_hh102_mapping.json', 'r') as f:
    activitiy_mapping = json.load(f)
activitiy_mapping

In [ ]:
cnn_act.shape

In [ ]:
cf = multilabel_confusion_matrix(cnn_act[-1115:], cnn_pred.T)
cf_dict = {}
for act, cf in zip(activitiy_mapping.keys(), cf):
    tp, fp, fn, tn = cf[0][0], cf[0][1], cf[1][0], cf[1][1]
    cf_dict[act] = {'tp':tp, 'fp':fp,'fn':fn, 'tn':tn, 'support': np.count_nonzero(np.array(cnn_act[-1115:]).T[activitiy_mapping[act]]), 'total': np.count_nonzero(np.array(cnn_act).T[activitiy_mapping[act]])}
pd.DataFrame(cf_dict)

In [ ]:
import pandas as pd
result = pd.read_csv('./result_new/BiLSTM-(fib_hh102)_mean_with_weight/report.txt', sep='\t')
top_k = result.sort_values(by='positive"F1').index[:7]
top_k
result.sort_values(by='positive"F1')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
loss = np.load('./result_new/BiLSTM-(fib_hh102)_mean_with_weight/losses.npy', allow_pickle=True)

for i in range(len(loss)):
    plt.plot(range(1, len(loss[i]) + 1), loss[i])
plt.xlim([0, 20])
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [ ]:
D = {u'Label1':26, u'Label2': 17, u'Label3':30}
count = np.count_nonzero(act.T == 1, axis=1)
plt.bar(range(len(activitiy_mapping)-1), list(count), align='center')
plt.xticks(range(len(activitiy_mapping)-1), list(activitiy_mapping.keys())[:-1], rotation=90)
# # for python 2.x:
# plt.bar(range(len(D)), D.values(), align='center')  # python 2.x
# plt.xticks(range(len(D)), D.keys())  # in python 2.x

plt.show()
